In [ ]:
import numpy as np
import math
from scipy.linalg import orth
from numpy.random import rand
from work.Optimize_W import Optimize_W
from work.Optimize_S import Optimize_S
from work.Optimize_alpha_k import Optimize_alpha_k
from work.generate_graphs import generate_graphs
from work.preprocessing import data_preprocess

In [ ]:
'''
数据集url
'''
from work.preprocessing import preprocess_normal_std
data_path = 'work/data/Indian_pines_corrected.mat'
labels_path = 'work/data/Indian_pines_gt.mat'
data_mat, labels_mat = preprocess_normal_std(data_path, labels_path)

In [ ]:

def SPCA_AMGLUSBS(data_path, labels_path, alpha, beta, gamma, d):
    '''
    X: 数据矩阵,论文中shape为[b,mn*mn],代码中实际为[mn*mn,b]
    d: 选择的波段子集中包含的波段个数
    '''
    W = 20
    H = 20
    X, y = data_preprocess(data_path, labels_path, W, H)
    
    mn = X.shape[0]
    n_neighbors = 20
    b = X.shape[1]

    #正则化子图Si
    Si = generate_graphs(X, n_neighbors)
    K = Si.shape[0]
   
    #初始化S
    S = np.sum(Si, axis=0) / K

    #初始化系数参数   
    gamma = 1 / K
    alpha_k = np.ones(K)
    alpha_k = alpha_k / K
    iteration = 0
    iter_max = 5
    W_init = orth(np.random.randn(b,d))

    while iteration < iter_max:
      
        # update W
        W = Optimize_W(X, W_init, S, d, alpha, beta, 1)

        # update S
        S = Optimize_S(X, W, Si, beta, gamma, alpha_k)
       
        # update alpha_k
        alpha_k = Optimize_alpha_k(Si, S)

        iteration = iteration + 1
    score = np.sum(W * W, axis=1)
    index_sorted = np.argsort(score)        #升序排列,需要逆序处理
    index_sorted = index_sorted[-d:][::-1]  #选择后d个

    return index_sorted

In [ ]:
# 稀疏PCA及自适应多图学习的评估
from work.estimate import eval_band_SVM
from work.estimate import eval_band_KNN
acc_s = 0
acc_k = 0
kappa_s = 0
kappa_k = 0
for i in range(10):
    #print('alpha:',alpha,'beta:',beta,'gamma:',gamma)
    # 提高alpha降低性能，提高beta有助于提高SVM，提高gamma有助于提高KNN
    sorted_index = SPCA_AMGLUSBS(data_path, labels_path,1,300,70,35)
 
    # gamma不能大
    data_mat_new, labels_mat = data_mat[:,sorted_index], labels_mat
    num = 20
    acc_svm = 0
    acc_knn = 0
    kappa_svm = 0
    kappa_knn = 0
    for o in range(num):
        a, b = eval_band_SVM(data_mat_new, labels_mat)
        c, d = eval_band_KNN(data_mat_new, labels_mat)
        acc_svm += a
        kappa_svm += b
        acc_knn += c
        kappa_knn += d
    acc_svm /= num
    acc_knn /= num
    kappa_svm /= num
    kappa_knn /= num
    acc_s += acc_svm
    acc_k += acc_knn
    kappa_s += kappa_svm
    kappa_k += kappa_knn
    print('acc by svm:', acc_svm)
    print('acc by knn:', acc_knn)
    print('kappa by svm:', kappa_svm)
    print('kappa by knn:', kappa_knn)
print(acc_s/10)    
print(acc_k/10)
print(kappa_s/10)
print(kappa_k/10)


In [28]:
# TRC-OC-FDPC
# NC-OC-MVPCA
from work.evaluate import evaluate_matlab
from work.preprocessing import preprocess_normalize
from work.preprocessing import preprocess_nonzero
from work.bands.Indian_pines.TRC_OC_FDPC import Bands_TRC_OC_FDPC
from work.bands.Indian_pines.NC_OC_MVPCA import Bands_NC_OC_MVPCA

# 这两个方法数据采取正则化预处理
data, labels = preprocess_normalize(data_path, labels_path)
a = Bands_NC_OC_MVPCA[15]
print(len(a))
evaluate_matlab(a, data, labels)

In [39]:
# MVPCA
from work.MVPCA import MVPCA
from scipy.io import loadmat
from work.evaluate import evaluate_python

# MVPCA的数据不能正则化预处理,否则影响分类精确度
data = loadmat(data_path)
index = list(data)[-1]
data = data[index]
data = data.reshape(-1,data.shape[2])
data = data.T

d = 50
band_index = MVPCA(data, d)
evaluate_python(band_index,data_mat,labels_mat)

In [53]:
# ISSC
from work.ISSC import ISSC_HSI
from scipy.io import loadmat
from work.estimate import eval_band_SVM
from work.estimate import eval_band_KNN
from work.preprocessing import preprocess_normalize

# 数据预处理部分只要正则化,不需要标准化
data, _ = preprocess_normalize(data_path, labels_path)
model = ISSC_HSI(40)
data_mat_new = model.predict(data)
num = 20
acc_svm = 0
acc_knn = 0
kappa_svm = 0
kappa_knn = 0
for i in range(num):
       a, b = eval_band_SVM(data_mat_new, labels_mat)
       c, d = eval_band_KNN(data_mat_new, labels_mat)
       acc_svm += a
       kappa_svm += b
       acc_knn += c
       kappa_knn += d
acc_svm /= num
acc_knn /= num
kappa_svm /= num
kappa_knn /= num
print('acc by svm:', acc_svm)
print('acc by knn:', acc_knn)
print('kappa by svm:', kappa_svm)
print('kappa by knn:', kappa_knn)

In [63]:
# SpaBS
from work.SpaBS import SpaBS
from work.estimate import eval_band_SVM
from work.estimate import eval_band_KNN
from work.preprocessing import preprocess_nonzero
from work.preprocessing import preprocess_normalize

data, _ = preprocess_normalize(data_path, labels_path)
model = SpaBS(50)
data_mat_new = model.predict(data)
num = 20
acc_svm = 0
acc_knn = 0
kappa_svm = 0
kappa_knn = 0
for i in range(num):
       a, b = eval_band_SVM(data_mat_new, labels_mat)
       c, d = eval_band_KNN(data_mat_new, labels_mat)
       acc_svm += a
       kappa_svm += b
       acc_knn += c
       kappa_knn += d
acc_svm /= num
acc_knn /= num
kappa_svm /= num
kappa_knn /= num
print('acc by svm:', acc_svm)
print('acc by knn:', acc_knn)
print('kappa by svm:', kappa_svm)
print('kappa by knn:', kappa_knn)

In [64]:
# all bands 
from work.estimate import eval_band_SVM
from work.estimate import eval_band_KNN
num = 20
acc_svm = 0
acc_knn = 0
kappa_svm = 0
kappa_knn = 0
for i in range(num):
       a, b = eval_band_SVM(data_mat_new, labels_mat)
       c, d = eval_band_KNN(data_mat_new, labels_mat)
       acc_svm += a
       kappa_svm += b
       acc_knn += c
       kappa_knn += d
acc_svm /= num
acc_knn /= num
kappa_svm /= num
kappa_knn /= num
print('acc by svm:', acc_svm)
print('acc by knn:', acc_knn)
print('kappa by svm:', kappa_svm)
print('kappa by knn:', kappa_knn)